In [ ]:
# ----------------------------------------------------
# PCA 사용 전후 K-Means 실루엣 점수 비교 코드
# (N_COMPONENTS 값 변경 시 모든 설명이 동적으로 변경됨)
# ----------------------------------------------------

import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
import warnings

# 경고 메시지 무시 (n_init 관련)
warnings.filterwarnings("ignore", category=FutureWarning)

# 데이터 준비 및 전처리
file_path = "CC GENERAL.csv"
df = pd.read_csv(file_path)
df_model = df.drop('CUST_ID', axis=1, errors='ignore')

# 결측치 처리: 평균값으로 대체
df_model.fillna(df_model.mean(), inplace=True)

# 데이터 표준화
scaler = StandardScaler()
X_scaled = scaler.fit_transform(df_model)

# K 값 설정
OPTIMAL_K = 4
# 🚨 변경 가능: PCA 주성분 개수 설정

N_COMPONENTS = 2 

print(f"기준 K 값: {OPTIMAL_K}")
print(f"PCA 축소 목표 차원: {N_COMPONENTS}")
print("-" * 60)

# ----------------------------------------------------
## 1. PCA 미사용 시 실루엣 점수
# ----------------------------------------------------

# K-Means 모델 학습 (원본 17차원 데이터 X_scaled 사용)
kmeans_original = KMeans(n_clusters=OPTIMAL_K, init='k-means++', max_iter=300, n_init=10, random_state=42)
labels_original = kmeans_original.fit_predict(X_scaled)

# 실루엣 점수 계산
score_original = silhouette_score(X_scaled, labels_original)

print(f"1. PCA 미사용 (원본 {X_scaled.shape[1]}차원) 실루엣 점수: {score_original:.4f}")

# ----------------------------------------------------
## 2. PCA 사용 시 실루엣 점수
# ----------------------------------------------------

# PCA 적용 
pca = PCA(n_components=N_COMPONENTS)
X_pca = pca.fit_transform(X_scaled)
cumulative_variance = pca.explained_variance_ratio_.sum() # 누적 분산 설명력 계산

# K-Means 모델 학습 (PCA 축소 데이터 X_pca 사용)
kmeans_pca = KMeans(n_clusters=OPTIMAL_K, init='k-means++', max_iter=300, n_init=10, random_state=42)
labels_pca = kmeans_pca.fit_predict(X_pca)

# 실루엣 점수 계산
score_pca = silhouette_score(X_pca, labels_pca)

print(f"2. PCA 사용 (축소된 {N_COMPONENTS}차원) 실루엣 점수: {score_pca:.4f}")
print(f"   (누적 분산 설명력: {cumulative_variance:.4f})") # 설명력 출력 추가
print("-" * 60)


## 3. 결과 비교
# ----------------------------------------------------

# if-elif-else 문을 함수 밖에서 바로 실행하여 결과 비교 출력

if score_pca > score_original:
    comparison = f"PCA({N_COMPONENTS}차원)를 사용했을 때 실루엣 점수가 ({score_original:.4f} -> {score_pca:.4f})로 더 높아, 군집 성능이 향상되었습니다. 이는 PCA가 노이즈를 제거하고 군집에 중요한 패턴을 강조했기 때문일 수 있습니다."
elif score_pca < score_original:
    comparison = f"PCA({N_COMPONENTS}차원)를 사용했을 때 실루엣 점수가 ({score_original:.4f} -> {score_pca:.4f})로 더 낮습니다. 이는 PCA 과정에서 군집 간의 경계를 구분하는 데 중요한 정보가 손실되었음을 의미할 수 있습니다."
else:
    comparison = "PCA 사용 전후의 실루엣 점수가 거의 동일합니다."

print("3. 최종 비교 결과:")
print(f"   PCA 미사용 점수: {score_original:.4f}")
print(f"   PCA 사용 점수: {score_pca:.4f}")
print(f"\n결론: {comparison}")